# 챗봇 기초(2)

## OpenAI API 활용

In [3]:
# pip install openai

### 함수 호출

### 대화 상태 관리

In [5]:
pip install OpenAIChatbot

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for OpenAIChatbot: filename=openaichatbot-1.0-py3-none-any.whl size=4652 sha256=64321b150e26834a5778f4b3de5e254bc315d5d7ddb6861a5ccfa5bc2bd03193
  Stored in directory: /Users/apple/Library/Caches/pip/wheels/cb/e6/9b/f89fe8a0bd1a623cbe1ea79bfbb59a3fc2855d3ccdf7380010
Successfully built OpenAIChatbot
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from openai import OpenAI
from dataclasses import dataclass, field
from typing import Dict, List, Any, Optional
from datetime import datetime
import uuid
from OpenAIChatbot import OpenAIChatbot
import os
import dotenv
from EntityExtractor import EntityExtractor


ModuleNotFoundError: No module named 'OpenAIChatbot'

## Agent

In [6]:
# pip install dotenv

In [24]:
import json
import time
from enum import Enum
from typing import List, Dict, Any,Optional
from dataclasses import dataclass
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")

In [26]:
class TaskStatus(Enum):
    """
    작업 상태를 나타내는 열거형 클래스
    """
    
    PENDING = "pending"
    RUNNING = "running"
    COMPLETED = "completed"
    FAILED = "failed"
    
@dataclass
class Task:
    """
    개별 작업을 나타내는 데이터 클래스
    """
    id: str
    description: str
    action: str
    parameters: Dict[str, Any]
    status: TaskStatus = TaskStatus.PENDING
    result: Optional[Any] = None
    error: Optional[str] = None
    
# 사용자의 목표를 분석하여 구체적인 작업 계획을 수립하는 에이전트
class PlanningAgent():
    def __init__(self, llm_client):
        self.llm_client = llm_client
        
        self.available_actions = [
            'web_search', 'calculate','send_email','save_file', 'analyze_data'
        ]
    def create_plan(self, user_goal: str) -> List[Task]:
        actions_description = """
        사용 가능한 액션들:
        - web_search: 웹에서 정보 검색 (parameters: {"query": "검색어"})
        - calculate: 수학 계산 (parameters: {"expression": "1+1"})
        - send_email: 이메일 발송 (parameters: {"to": "email", "subject": "제목", "body": "내용"})
        - save_file: 파일 저장 (parameters: {"filename": "파일명", "content": "내용"})
        - analyze_data: 데이터 분석 (parameters: {"data_source": "데이터소스", "analysis_type": "분석타입"})
        """

        system_prompt = f"""
        당신은 전문적인 계획 수립 에이전트입니다.
        사용자 목표를 분석하고 단계별 작업 계획을 세우세요.
        {actions_description}
        중요: 반드시 위의 5가지 액션 중에서만 선택해야 합니다.
        각 작업은 구체적이고 실행 가능해야 합니다.

        응답 형식 (반드시 JSON 형태로):

        [
        {{
            "id": "task_1",
            "description": "작업 설명",
            "action": "web_search",
            "parameters": {{"query": "검색어"}}
        }}
        ]
        """
        
        try: 
            response = self.llm_client.chat.completions.create(
                model = 'gpt-4o-mini',
                messages = [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"목표:{user_goal}"}
                ],
                temperature = 0.1
            )
            
            content = response.choices[0].message.content.strip()
            
            if "```json" in content:
                content = content.split("```json")[1].split("```")[0].strip()
            elif "```" in content:
                content = content.split("```")[1].strip()

            # JSON 문자열을 파이썬 객체로 파싱
            tasks_data = json.loads(content)
            tasks = []

            # 각 작업 데이터를 Task 객체로 변환
            for i, task_data in enumerate(tasks_data):
                # 액션 유효성 검사
                action = task_data.get("action", "")
                if action not in self.available_actions:
                    print(f"경고: 유효하지 않은 액션 '{action}'을 'web_search'로 변경")
                    action = "web_search"
                    # 기본 검색 쿼리로 작업 설명 사용
                    task_data["parameters"] = {"query": task_data.get("description", "")}

                # Task 객체 생성 (Task 클래스는 dataclass로 사전 정의)
                task = Task(
                    id=task_data.get("id", f"task_{i+1}"),
                    description=task_data.get("description", ""),
                    action=action,
                    parameters=task_data.get("parameters", {})
                )
                tasks.append(task)
            return tasks
        
        except Exception as e: 
            print(f"계획 수립 중 오류 발생 {e}")
            
            return [
                Task(
                    id = 'task_1',
                    description = f"'{user_goal}'에 대한 정보 검색",
                    action = "web_search",
                    parameters = {"query": user_goal}
                )
            ]
            
class ExecutionEngine:
    def __init__(self):
        
        self.actions = {
            'web_search': self._web_search, 
            'calculate': self._calculate,
            'send_email' : self._send_email,
            'save_file': self._save_file,
            'analyze_data': self._analyze_data
        }
        
    def execute_task(self,task: Task) -> Task :
        try:
        # 작업 상태를 실행 중으로 변경
            task.status = TaskStatus.RUNNING

            # 액션이 등록된 함수인지 확인
            if task.action in self.actions:
                # 해당 액션 함수 실행
                # self.actions[task.action]은 함수 객체 반환
                result = self.actions[task.action](task.parameters)
                task.result = result
                task.status = TaskStatus.COMPLETED
            else:
                # 등록되지 않은 액션에 대한 오류 처리
                task.error = f"알 수 없는 액션: {task.action}"
                task.status = TaskStatus.FAILED

        except Exception as e:
            # 실행 중 발생한 예외 처리
            # str(e)를 사용하는 이유: 예외 객체를 문자열로 변환하여 저장
            task.error = str(e)
            task.status = TaskStatus.FAILED

        return task

            
    def _web_search(self, params: Dict[str,Any]) -> Dict[str, Any]:
        query = params.get('query',"")
        print(f"웹 검색 : {query}")
        
        return {
        "query": query,
        "results": [
            {"title": f"{query} 관련 정보 1", "url": "https://example1.com"},
            {"title": f"{query} 관련 정보 2", "url": "https://example2.com"},
            {"title": f"{query} 관련 정보 3", "url": "https://example3.com"}
        ]
    }
    
    def _calculate(self, params: dict) -> dict:
        expression = params.get("expression", "0")
        try:
            # 보안상 위험한 키워드 검사
            # any()를 사용하는 이유: 하나라도 포함되면 True 반환
            if any(char in expression for char in ['import', 'exec', 'eval', '__']):
                raise ValueError("보안상 허용되지 않는 수식입니다.")

            result = eval(expression)
            print(f"계산:{expression} = {result}")
            
            return {"expression": expression, 'result': result}
        
        except Exception as e:
            raise ValueError(f"error: str{e}")
    
    def _send_email(self, params : Dict[str,Any]) -> Dict[str,Any]:
        
        to = params.get("to", "")
        subject = params.get("subject","")
        body = params.get("body", "")
        
        print(f"이메일 발송 : {to} / {subject}")
        
        return {
            "status" : "sent",
            "to": to,
            "subject": subject,
            "timestamp" : time.time()
        }
     
    def _save_file(self, params: dict) -> dict:
  
        filename = params.get("filename", "output.txt")
        content = params.get("content", "")

        try:
            # 파일을 UTF-8 인코딩으로 저장
            # with 문을 사용하면 파일이 자동으로 닫힘
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(content)

            print(f"파일 저장: {filename}")

            return {
                "filename": filename,
                "size": len(content),  # 바이트 단위가 아닌 문자열 길이 기준
                "saved": True
            }
        except Exception as e:
            # 파일 저장 실패 시 예외 발생 및 메시지 반환
            # f-string을 사용하는 이유: 오류 메시지에 원인 포함
            raise Exception(f"파일 저장 실패: {str(e)}")
        
    def _analyze_data(self, params: dict) -> dict:

        data_source = params.get("data_source", "")
        analysis_type = params.get("analysis_type", "basic")

        print(f"데이터 분석: {data_source} / {analysis_type}")

        # f-string을 사용하여 동적 결과 생성
        return {
            "data_source": data_source,
            "analysis_type": analysis_type,
            "summary": f"{data_source}에 대한 {analysis_type} 분석 완료",
            "insights": [
                f"{data_source}의 주요 트렌드 파악",
                f"{analysis_type} 분석을 통한 인사이트 도출",
                "향후 개선 방향 제시"
            ]
        }


class AdvancedAgent:
    def __init__(self, llm_client):
        """
        AdvancedAgent 초기화 메서드
        """
        self.planner = PlanningAgent(llm_client)   # 계획 수립 담당
        self.executor = ExecutionEngine()          # 작업 실행 담당
        self.llm_client = llm_client               # LLM 클라이언트 참조 보관

    def solve_problem(self, user_goal: str) -> dict:
        """
        사용자 목표에 대한 전체 문제 해결 프로세스를 수행하는 메서드
        """
        print(f"목표: {user_goal}")

        # 1단계: 계획 수립
        print("1단계: 계획 수립")
        tasks = self.planner.create_plan(user_goal)

        # 계획 수립 실패 시 즉시 종료
        if not tasks:
            return {"error": "계획 수립에 실패했습니다.", "success": False}

        # 생성된 작업들 출력
        print(f"생성된 작업: {len(tasks)}개")
        for task in tasks:
            # 작업 설명과 함께 사용될 액션도 표시
            print(f"- {task.description} [{task.action}]")

        # 2단계: 작업 실행
        print(f"\n2단계: 작업 실행")
        completed_tasks = 0  # 완료된 작업 카운터

        # enumerate 사용: 인덱스와 값 필요, 1부터 시작
        for i, task in enumerate(tasks, 1):
            print(f"작업 {i}/{len(tasks)}: {task.description}")

            # 작업 실행 및 결과로 원본 태스크 업데이트
            tasks[i-1] = self.executor.execute_task(task)

            # 실행 결과에 따른 피드백 출력
            if task.status == TaskStatus.FAILED:
                print(f"실패: {task.error}")
            else:
                print(f"완료")
                completed_tasks += 1

        # 3단계: 결과 정리 및 통계 계산
        success_rate = (completed_tasks / len(tasks)) * 100
        goal_achieved = completed_tasks > 0   # 하나라도 성공시 True

        print(f"\n결과 요약")
        print(f"성공률: {success_rate:.1f}%")
        print(f"완료: {completed_tasks}/{len(tasks)}")

        # 4단계: 결과 요약 문자열 생성
        if completed_tasks > 0:
            results_summary = []
            for task in tasks:
                if task.status == TaskStatus.COMPLETED:
                    results_summary.append(f"성공 ({task.description}): 완료")
                else:
                    results_summary.append(f"실패 ({task.description}): 실패")
            
            final_result = " \n ".join(results_summary)
        else:
            final_result = "모든 작업 실패"
            
        return {
            "goal": user_goal,                # 원래 목표 보존
            "tasks": tasks,                   # 모든 작업 객체들(결과 포함)
            "success_rate": success_rate,     # 성공률 (0-100)
            "completed_tasks": completed_tasks,   # 성공한 작업 수
            "total_tasks": len(tasks),        # 전체 작업 수
            "success": goal_achieved,         # 전체적 성공 여부
            "summary": final_result           # 텍스트 요약
                
        }
        
if __name__ == "__main__":
    if not api_key:
        print("OPENAI_API_KEY가 설정되지 않았습니다.")
        print("환경 변수를 설정하거나 .env 파일을 확인하세요.")
        exit(1)    # 오류 코드 1로 프로그램 종료

    try:
        # OpenAI 클라이언트 생성 및 에이전트 초기화
        client = OpenAI(api_key=api_key)
        agent = AdvancedAgent(client)

        # 테스트용 목표들
        test_goals = [
            "파이썬 프로그래밍 학습 자료 찾기",   # 정보 검색 중심
            "회사 매출 데이터 분석 보고서 생성",   # 분석 및 문서 작성
            "프로젝트 진행 상황 팀에 공유"       # 커뮤니케이션 중심
        ]

        # 다양한 유형의 목표들을 테스트하며 서로 다른 목표 선정
        for goal in test_goals:
            result = agent.solve_problem(goal)
            print(f"\n최종 결과:{'성공' if result['success'] else '실패'}")
            print('*'*50)
            
    except Exception as e:
        print("에이전트 실행 중 예외 발생:", str(e))
        print("API 키나 네트워크 연결을 확인해 주세요")


목표: 파이썬 프로그래밍 학습 자료 찾기
1단계: 계획 수립
생성된 작업: 1개
- 파이썬 프로그래밍 학습 자료를 찾기 위해 웹 검색을 수행합니다. [web_search]

2단계: 작업 실행
작업 1/1: 파이썬 프로그래밍 학습 자료를 찾기 위해 웹 검색을 수행합니다.
웹 검색 : 파이썬 프로그래밍 학습 자료
완료

결과 요약
성공률: 100.0%
완료: 1/1

최종 결과:성공
**************************************************
목표: 회사 매출 데이터 분석 보고서 생성
1단계: 계획 수립
생성된 작업: 3개
- 회사의 매출 데이터 소스를 확인하기 위해 웹에서 관련 정보를 검색합니다. [web_search]
- 확인된 매출 데이터 소스를 기반으로 데이터 분석을 수행합니다. [analyze_data]
- 분석 결과를 바탕으로 보고서를 작성합니다. [save_file]

2단계: 작업 실행
작업 1/3: 회사의 매출 데이터 소스를 확인하기 위해 웹에서 관련 정보를 검색합니다.
웹 검색 : 회사 매출 데이터 소스
완료
작업 2/3: 확인된 매출 데이터 소스를 기반으로 데이터 분석을 수행합니다.
데이터 분석: 확인된 매출 데이터 소스 / 매출 추세 분석
완료
작업 3/3: 분석 결과를 바탕으로 보고서를 작성합니다.
파일 저장: 매출_분석_보고서.txt
완료

결과 요약
성공률: 100.0%
완료: 3/3

최종 결과:성공
**************************************************
목표: 프로젝트 진행 상황 팀에 공유
1단계: 계획 수립
생성된 작업: 3개
- 프로젝트 진행 상황을 요약하여 이메일로 팀에 공유하기 위한 내용 작성 [web_search]
- 팀원들의 이메일 주소 수집 [web_search]
- 작성한 내용을 바탕으로 이메일 발송 [send_email]

2단계: 작업 실행
작업 1/3: 프로젝트 진행 상황을 요약하여 이메일로 팀에 공유하기 위한 내용 작성
웹 검색 : 프로젝트 진행